In [12]:
%pip install -r ../requirements.txt
path = '../path.txt'
with open(path, 'r') as archivo:
    for linea in archivo.readlines():
        exec(linea.strip(), globals())

  Using cached branca-0.7.2-py3-none-any.whl (25 kB)
  Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Using cached ipython-8.24.0-py3-none-any.whl (816 kB)
  Using cached jupyter_client-8.6.1-py3-none-any.whl (105 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-win_amd64.whl (7.7 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.2
    Uninstalling matplotlib-3.8.2:
      Successfully uninstalled matplotlib-3.8.2
  Attempting uninstall: jupyter_client
    Found existing installation: jupyter_client 8.3.0
    Uninstalling jupyter_client-8.3.0:
      Successfully uninstalled jupyter_client-8.3.0
  Attempting uninstall: branca
    Found existing installation: branca 0.7.1
    Uninstalling branca-0.7.1:
      Successfully uninstalled branca-0.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 8.14.0
    Uninstalling ipython-8.14.0:
      Successfully uninstalled ipython-8.14.0
  Attempting uninstall: ipykernel
    Fou


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from joblib import dump
import warnings
warnings.filterwarnings("ignore")

In [14]:
df = pd.read_parquet(dataset_entrenamiento_distancia)

In [15]:
df['distancia'][(df['distancia'] >= 0) & (df['distancia'] <= 4)] = 0
df['distancia'][(df['distancia'] > 4) & (df['distancia'] <= 8)] = 1
df['distancia'][(df['distancia'] > 8) & (df['distancia'] <= 12)] = 2
df['distancia'][(df['distancia'] > 12) & (df['distancia'] <= 16)] = 3
df['distancia'][(df['distancia'] > 16) & (df['distancia'] <= 20)] = 4

In [16]:
df['distancia'].unique()

array([1., 0., 2., 4., 3.])

In [17]:
df['hora'].unique()

array([18, 12, 21, 10, 13,  7, 19, 23, 14, 15, 16,  6, 20, 22, 11,  8, 17,
        1,  0,  9,  3,  2,  5,  4])

In [18]:
df['hora'][(df['hora'] >= 0) & (df['hora'] <= 3)] = 0
df['hora'][(df['hora'] > 3) & (df['hora'] <= 6)] = 1
df['hora'][(df['hora'] > 6) & (df['hora'] <= 9)] = 2
df['hora'][(df['hora'] > 9) & (df['hora'] <= 12)] = 3
df['hora'][(df['hora'] > 12) & (df['hora'] <= 15)] = 4
df['hora'][(df['hora'] > 15) & (df['hora'] <= 18)] = 5
df['hora'][(df['hora'] > 18) & (df['hora'] <= 21)] = 6
df['hora'][(df['hora'] > 21) & (df['hora'] < 24)] = 7

In [19]:
df = df.drop('dia_del_mes', axis=1)

In [20]:
df.shape

(840000, 4)

In [21]:
def crear_modelo(data:pd.DataFrame, modelo:object, scaler:object, label:str, split:float=0.2, **kwargs) -> object:

    """
    Función para crear distintos modelos.

    Args:

        - data: df utilizado (objeto)

        - modelo: modelo seleccionado (objeto)

        - scaler: scaler seleccionado (objeto)

        - split: porcentaje del dataset que se usa para test (float, default=0.2)

        - label: nombre de la variable objetivo (str)

        - **kwargs_modelo: hiperparámetros específicos de cada modelo

            ##  Regresión logística:

                - C  (default = 1.0): parámetro de regularización inversa. Controla la fuerza de regularización; valores más pequeños especifican una regularización más fuerte (float)
                - solver  (default = 'lbfgs'): algoritmo a utilizar en la optimización. Opciones disponibles: 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga' (str)
                - max_iter  (default = 100): número máximo de iteraciones permitidas para la convergencia del algoritmo (int)

            ##  Árbol de decisión:

                - criterion  (default = 'gini'): función para medir la calidad de la división. Opciones disponibles: 'gini', 'entropy' (str)
                - max_depth  (default = None): profundidad máxima del árbol. Si es None, los nodos se expanden hasta que todas las hojas sean puras o contengan menos de min_samples_split muestras (int | None)
                - min_samples_split  (default = 2): número mínimo de muestras necesarias para dividir un nodo interno. Este parámetro controla el número mínimo de muestras que se requieren en un nodo para que pueda ser dividido en uno o más nodos hijos. Si una división resulta en un número menor de muestras que el valor especificado para min_samples_split, la división se ignorará y el nodo se considerará una hoja. (int | float)

            ##  Random Forest:

                - n_estimators  (default = 100): número de árboles en el bosque (int)
                - max_depth  (default = None): profundidad máxima de cada árbol en el bosque (int | None)
                - min_samples_split  (default = 2): número mínimo de muestras necesarias para dividir un nodo interno (int | float)

            ##  Naive Bayes (Gaussian):

                - no tiene hiperparámetros específicos

    Return:

        - modelo entrenado (objeto)

    Notas:

    Al llamar a la función, el scaler debe ir con paréntesis y, si es necesario pasar algún argumento (como MinMaxScaler(feature_range=(0, 10))), se pasa al invocar la función.

    El modelo, en cambio, no va con paréntesis, ya que recibe sus parámetros desde **kwargs. Está como objeto, pero sin instanciar.

    Ejemplo:

    modelo_1 = crear_modelo(LogisticRegression, StandardScaler(), split=0.15, label='isFraud', max_iter=150)
    """

    try:
        modelo_instanciado = modelo(**kwargs)
    except:
        raise 'Error: argumentos no válidos.'
    scaler_instanciado = scaler

    X = data.drop(label, axis=1)
    y = data[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)

    if scaler != None:
        X_train_scaled = scaler_instanciado.fit_transform(X_train)
        X_test_scaled = scaler_instanciado.transform(X_test)
        modelo_instanciado.fit(X_train_scaled, y_train)
    else:
        modelo_instanciado.fit(X_train, y_train)

    predicciones_train = modelo_instanciado.predict(X_train_scaled)
    predicciones = modelo_instanciado.predict(X_test_scaled)
    metricas_train = accuracy_score(y_train, predicciones_train)
    metricas = accuracy_score(y_test, predicciones)

    print(f'\n\nModelo: {str(modelo)}\nScaler: {scaler}\nCantidad de registros de prueba: {int(len(data)*split)}')
    print(f'\nMétricas en entrenamiento: {round(metricas_train, 2)}\n')
    print(f'\nMétricas en prueba: {round(metricas, 2)}\n')
    print(f'Reporte de entrenamiento: \n{classification_report(y_train, predicciones_train)}\n\n')
    print(f'Reporte de test: \n{classification_report(y_test, predicciones)}\n\n')

    return modelo

In [24]:
random_forest = crear_modelo(df, RandomForestClassifier, StandardScaler(), label='distancia', n_estimators=200, max_depth=25, min_samples_split=5)



Modelo: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Scaler: StandardScaler()
Cantidad de registros de prueba: 168000

Métricas en entrenamiento: 0.83


Métricas en prueba: 0.82

Reporte de entrenamiento: 
              precision    recall  f1-score   support

         0.0       0.85      0.99      0.91    535594
         1.0       0.53      0.09      0.16     79643
         2.0       0.61      0.33      0.43     31739
         3.0       0.51      0.05      0.10     10184
         4.0       0.58      0.58      0.58     14840

    accuracy                           0.83    672000
   macro avg       0.61      0.41      0.44    672000
weighted avg       0.79      0.83      0.78    672000



Reporte de test: 
              precision    recall  f1-score   support

         0.0       0.84      0.98      0.91    133726
         1.0       0.42      0.07      0.13     19981
         2.0       0.57      0.31      0.41      8050
         3.0       0.26      0.03      0.05      2538

In [25]:
dump(random_forest, modelo_distancia_entrenado)

['../modelos_ML/modelos_entrenados/modelo_distancia_entrenado.joblib']